#### Let's Begin by importing both the libs sklearn and cuml to measure the performance difference

Starting with the basics lets try Random Forest Classifier for checking heardisease

In [16]:
!nvidia-smi

/bin/bash: /home/yashdhingra/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Sun Aug  6 22:31:33 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.04              Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        On  | 00000000:01:00.0  On |                  N/A |
|  0%   50C    P8              21W / 170W |   1757MiB / 12288MiB |      4%      Default |
|                                         

In [2]:
import cudf as df
import pandas as pd

## Let's import the data now

In [3]:
data_df=df.read_csv("heart-disease.csv")
data_pd=pd.read_csv("heart-disease.csv")

In [4]:
data_df

,﻿age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


### getting training and test data split ready 

In [7]:
from sklearn.model_selection import train_test_split as tts_sk
from cuml.model_selection import train_test_split as tts_ml

In [12]:
x=data_df.drop('target',axis=1)
y=data_df['target']

X=data_pd.drop('target',axis=1)
Y=data_pd['target']

In [13]:
x_train,x_test,y_train,y_test = tts_ml(x,y,test_size=0.2)

In [15]:
X_train,X_test,Y_train,Y_test = tts_sk(X,Y,test_size=0.2)

### Importing Models 

In [19]:
from cuml.ensemble import RandomForestClassifier as rfc_ml
from sklearn.ensemble import RandomForestClassifier as rfc_sk

### Fittin Data

In [20]:
clf_ml=rfc_ml()
clf_sk=rfc_sk()

In [24]:
from cuml.model_selection import GridSearchCV as GridSearchCV_ml
from sklearn.model_selection import GridSearchCV as GridSearchCV_sk


import cupy as py 
import numpy as np

In [23]:
param_grid = {'n_estimators': [i for i in range(10, 100, 10)]}

# Setup the grid search
grid = GridSearchCV_ml(clf_ml,
                    param_grid,
                    cv=5
                    ,verbose=True)

# Fit the grid search to the data
grid.fit(x, y)

# Find the best parameters
grid.best_params_

TypeError: Implicit conversion to a host NumPy array via __array__ is not allowed, To explicitly construct a GPU matrix, consider using .to_cupy()
To explicitly construct a host matrix, consider using .to_numpy().